In [8]:
from itertools import combinations
from math import factorial

import pandas as pd
from geopandas import GeoDataFrame, sjoin
from colocationpatterns.sample_data import generate_sample_data

In [48]:
class ColocationMiner:

    def __init__(self, data: GeoDataFrame, feature_type_column: str, feature_type_unique_id_column: str, neighbourhood: float):

        self.neighbourhood = neighbourhood
        self.K = data[feature_type_column].nunique()
        self.tables = [feature_type_data for feature_name, feature_type_data in data.groupby(by=feature_type_column, as_index=False)]
        self.table_ids_iterator = iter(range(0, sum(((factorial(self.K))//(factorial(i)*factorial(self.K-i)) for i in range(1, self.K+1)))))
        self.k_level_tables_ids = {1: [next(self.table_ids_iterator) for _ in range(self.K)]}

    def calculate_participation_ratio(self):

        pass

    def calculate_participation_index(self):

        pass

    def __call__(self):

        for k in range(2, self.K+1):
            self.k_level_tables_ids[k] = list()            
            for table_comb_ids in combinations(self.k_level_tables_ids[1], k):
                print(f'Creating table k_level = {k} for tables {table_comb_ids}')
               
                table = pd.concat(
                    [sjoin(self.tables[ti], self.tables[tj], how='inner', predicate='dwithin', distance=self.neighbourhood)
                        for ti, tj in zip(table_comb_ids[0::-1], table_comb_ids[1::])
                    ]
                )
                # TODO: Nie działa dla k >=3, tabele źle się łączą!!!
                self.tables.append(table)
            
                self.k_level_tables_ids[k].append(next(self.table_ids_iterator))
            
       
            

In [49]:
data = generate_sample_data()

In [50]:
cm = ColocationMiner(data, 'spatial_feature_type', 'instance_id', 3)

In [51]:
cm()

Creating table k_level = 2 for tables (0, 1)
Creating table k_level = 2 for tables (0, 2)
Creating table k_level = 2 for tables (1, 2)
Creating table k_level = 3 for tables (0, 1, 2)


In [25]:
cm.k_level_tables_ids

{1: [0, 1, 2], 2: [3, 4, 5], 3: [6]}

In [63]:
cm.tables[6]

,instance_id_left,spatial_feature_type_left,geometry,index_right,instance_id_right,spatial_feature_type_right
0,1,A,POINT (1.75 1.75),4,1,B
1,2,A,POINT (5.45 3.75),7,4,B
2,3,A,POINT (8.5 3.25),7,4,B
